In [9]:
import pandas as pd

# 1. Загрузка данных
file_path = './Data.xlsx'  # путь к вашему файлу
df = pd.read_excel(file_path)

# 2. Преобразование данных
df['Дата'] = pd.to_datetime(df['Дата'], format="%d.%m.%Y", dayfirst=True)  # Указываем формат даты

# Определяем текущую дату для расчета давности
today_date = dt.datetime.today()

# 3. Расчет RFM метрик

# Давность (Recency): количество дней с последней покупки
recency_df = df.groupby('ID').agg({'Дата': lambda x: (today_date - x.max()).days}).reset_index()
recency_df.columns = ['ID', 'Recency']

# Чистота (Frequency): количество покупок
frequency_df = df.groupby('ID').agg({'Дата': 'count'}).reset_index()
frequency_df.columns = ['ID', 'Frequency']

# Деньги (Monetary): общая сумма покупок
monetary_df = df.groupby('ID').agg({'Продажи': 'sum'}).reset_index()
monetary_df.columns = ['ID', 'Monetary']

# Объединение всех RFM метрик в один датафрейм
rfm_df = recency_df.merge(frequency_df, on='ID').merge(monetary_df, on='ID')

# 4. Сегментация клиентов

# Сегментация по Давности
rfm_df['Recency_Segment'] = rfm_df['Recency'].apply(lambda x: 1 if x <= 30 else (2 if x <= 180 else 3))

# Сегментация по Чистоте
rfm_df['Frequency_Segment'] = rfm_df['Frequency'].apply(lambda x: 1 if x > 5 else (2 if 3 <= x <= 5 else 3))

# Сегментация по Деньгам
rfm_df['Monetary_Segment'] = rfm_df['Monetary'].apply(lambda x: 1 if x > 3000 else (2 if 1500 <= x <= 3000 else 3))

# 5. Формирование RFM сегмента
rfm_df['RFM_Segment'] = rfm_df['Recency_Segment'].map(str) + rfm_df['Frequency_Segment'].map(str) + rfm_df['Monetary_Segment'].map(str)

# 6. Словарь для сопоставления сегментов , статусов и рекомендаций к действию!!!!!!!
status_mapping = {
    '133': 'Новичок',
    '132': 'Новичок',
    '131': 'Киты',
    '123': 'Лояльные',
    '122': 'Лояльные',
    '121': 'Киты',
    '113': 'Лояльные',
    '112': 'Лояльные',
    '111': 'Ядро',
    '233': 'Спящие',
    '232': 'Спящие',
    '231': 'Спящие',
    '223': 'Спящие',
    '222': 'Спящие',
    '221': 'Спящие',
    '213': 'Спящие',
    '212': 'Спящие',
    '211': 'Спящие',   
    '333': 'Потеряные',
    '332': 'Уходящие',
    '331': 'Уходящие',
    '323': 'Уходящие',
    '322': 'Уходящие',
    '321': 'Уходящие',
    '313': 'Уходящие',
    '312': 'Уходящие',
    '311': 'Уходящие'
}

recommendation_mapping = {
    '133': 'Отправляем welcome-цепочку с описанием преимуществ, ответами на частые вопросы, обучающие материалы',
    '132': 'Отправляем welcome-цепочку с описанием преимуществ, ответами на частые вопросы, обучающие материалы',
    '131': 'В дополнение к welcome-цепочке отправляем им интересное предложение, чтобы удержать интерес',
    '123': 'Нужно попытаться увеличить средний чек допродажей. Отправляем им рассылки с сопутствующими товарами',
    '122': 'Нужно попытаться увеличить средний чек допродажей. Отправляем им рассылки с сопутствующими товарами',
    '121': 'Эти клиенты и так хорошо покупают, не стоит утомлять их лишними рассылками. Отправляем только обычные рассылки',
    '113': 'Можно попытаться увеличить сумму чека допродажей. Отправляем им рассылки с сопутствующими товарами',
    '112': 'Наши самые хорошие клиенты, можно попробовать увеличить сумму чека. Отправляем им специальные предложения, сопутствующие товары',
    '111': 'Отправляем им особые предложения, например, участие в программе лояльности — приглашение в клуб любимых клиентов',
    '233': 'Эти клиенты ещё помнят о нас. Нужно попробовать расшевелить их стимулирующими акциями. Отправляем им рассылки с интересными акциями и выгодными предложениями.',
    '232': 'Эти клиенты ещё помнят о нас. Нужно попробовать расшевелить их стимулирующими акциями. Отправляем им рассылки с интересными акциями и выгодными предложениями.',
    '231': 'Эти клиенты ещё помнят о нас. Нужно попробовать расшевелить их стимулирующими акциями. Отправляем им рассылки с интересными акциями и выгодными предложениями.',
    '223': 'Эти клиенты ещё помнят о нас. Нужно попробовать расшевелить их стимулирующими акциями. Отправляем им рассылки с интересными акциями и выгодными предложениями.',
    '222': 'Эти клиенты ещё помнят о нас. Нужно попробовать расшевелить их стимулирующими акциями. Отправляем им рассылки с интересными акциями и выгодными предложениями.',
    '221': 'Эти клиенты ещё помнят о нас. Нужно попробовать расшевелить их стимулирующими акциями. Отправляем им рассылки с интересными акциями и выгодными предложениями.',
    '213': 'Ещё не так давно они часто покупали у вас, а сейчас перестали. Нужно обязательно узнать, в чём дело, и попытаться вернуть их. Отправляем им рассылку с интересными акциями и рассылку с опросом.',
    '212': 'Ещё не так давно они часто покупали у вас, а сейчас перестали. Нужно обязательно узнать, в чём дело, и попытаться вернуть их. Отправляем им рассылку с интересными акциями и рассылку с опросом.',
    '211': 'Ещё не так давно они часто покупали у вас, а сейчас перестали. Нужно обязательно узнать, в чём дело, и попытаться вернуть их. Отправляем им рассылку с интересными акциями и рассылку с опросом.' ,
    '333': 'Скорее всего, эти клиенты уже ушли от нас, и не стоит тратить слишком больших усилий на этот сегмент. Отправляем автоматическую цепочку реактивации.',
    '332': 'Скорее всего, эти клиенты уже ушли от нас, и не стоит тратить слишком больших усилий на этот сегмент. Отправляем автоматическую цепочку реактивации.',
    '331': 'Скорее всего, эти клиенты уже ушли от нас, и не стоит тратить слишком больших усилий на этот сегмент. Отправляем автоматическую цепочку реактивации.',
    '323': 'Эти клиенты сделали у нас несколько покупок, а потом перестали покупать. Их можно попытаться вернуть и обязательно получить обратную связь, почему они от нас ушли. Отправляем реактивационную рассылку с опросом.',
    '322': 'Эти клиенты сделали у нас несколько покупок, а потом перестали покупать. Их можно попытаться вернуть и обязательно получить обратную связь, почему они от нас ушли. Отправляем реактивационную рассылку с опросом.',
    '321': 'Эти клиенты сделали у нас несколько покупок, а потом перестали покупать. Их можно попытаться вернуть и обязательно получить обратную связь, почему они от нас ушли. Отправляем реактивационную рассылку с опросом.',
    '313': 'Этих клиентов обязательно нужно попытаться вернуть. Отправляем им реактивационные письма, запрашиваем обратную связь, предлагаем бонусы, программы лояльности.',
    '312': 'Этих клиентов обязательно нужно попытаться вернуть. Отправляем им реактивационные письма, запрашиваем обратную связь, предлагаем бонусы, программы лояльности.',
    '311': 'Этих клиентов обязательно нужно попытаться вернуть. Отправляем им реактивационные письма, запрашиваем обратную связь, предлагаем бонусы, программы лояльности.'
}
# 7. Присвоение статусов на основе RFM сегмента
rfm_df['Customer_Status'] = rfm_df['RFM_Segment'].map(status_mapping)

# 8 Добавляем рекомендации на основе RFM сегмента
rfm_df['Recommendation'] = rfm_df['RFM_Segment'].map(recommendation_mapping)

# 9 Сохранение результатов в новый Excel файл
rfm_df.to_excel('RFM_Status_Analysis.xlsx', index=False)

print(rfm_df.head())

   ID  Recency  Frequency  Monetary  Recency_Segment  Frequency_Segment  \
0   1      209          1      4359                3                  3   
1   2      160          3      4129                2                  2   
2   3      108          4      9189                2                  2   
3   4      101          3      4620                2                  2   
4   5      235          1      4540                3                  3   

   Monetary_Segment RFM_Segment Customer_Status  \
0                 1         331        Уходящие   
1                 1         221          Спящие   
2                 1         221          Спящие   
3                 1         221          Спящие   
4                 1         331        Уходящие   

                                      Recommendation  
0  Скорее всего, эти клиенты уже ушли от нас, и н...  
1  Эти клиенты ещё помнят о нас. Нужно попробоват...  
2  Эти клиенты ещё помнят о нас. Нужно попробоват...  
3  Эти клиенты ещё пом